In [128]:
import pandas as pd
import geopandas as gpd
import numpy as np 
import bokeh.plotting as bpl
import bokeh.models as bmd
import bokeh.layouts as bly
from bokeh.io import export_png
import json

In [79]:
bpl.output_notebook()

Loading BokehJS ...

In [126]:
alcaldias = gpd.read_file("../datos/alcaldias/09mun.shp")

In [139]:
geosource = bmd.GeoJSONDataSource(geojson = alcaldias.to_crs("EPSG:4326").to_json())

In [170]:
celdas = pd.read_csv("../salida/celdas_agebs.csv")
data1 = pd.read_csv("../salida/cemp_mpio_1.csv")
sector = "cemp_mpio"
eta = 1

In [171]:
celdas["dist_to_center"] = ((celdas["x"]-celdas["x"].mean())*(celdas["x"]-celdas["x"].mean()) + (celdas["y"]-celdas["y"].mean())*(celdas["y"]-celdas["y"].mean())).apply(np.sqrt)

In [172]:
colores = ['#b2182b','#f4a582','#92c5de','#2166ac']
colores.reverse()

In [173]:
celdas["color"] = pd.qcut(celdas["dist_to_center"],len(colores),labels=colores)

In [174]:
data1.index = data1.index.astype(str)

In [175]:
simulacion = data1.T.reset_index().rename(columns={"index": "CVE"})

In [176]:
p0 = bpl.figure(title="Evolución de tamaños, eta = " + str(eta), plot_width=1000, plot_height=500, toolbar_location=None)

In [177]:
for col in data1.columns:
    color = celdas.set_index("CVE").loc[col]["color"]
    p0.line(x=range(1,len(data1)+1),y=data1[col], color = color, line_width=2)

In [178]:
p0.xaxis.axis_label = 'Iteración'
p0.yaxis.axis_label = 'Tamaño'

In [179]:
bpl.show(p0)

In [180]:
iteracioni = 0
instantei = pd.merge(celdas,simulacion[["CVE",str(iteracioni)]].rename(columns={str(iteracioni):"size"}),on="CVE")
instantei["radio"] = instantei["size"].apply(np.sqrt)
iteracionf = 199
instantef = pd.merge(celdas,simulacion[["CVE",str(iteracionf)]].rename(columns={str(iteracionf):"size"}),on="CVE")
instantef["radio"] = instantef["size"].apply(np.sqrt)

In [181]:
pi = bpl.figure(title="Simulación, iteración " + str(iteracioni), toolbar_location=None, plot_width=500, plot_height=600)
pf = bpl.figure(title="Simulación, iteración " + str(iteracionf), toolbar_location=None, plot_width=500, plot_height=600)

In [182]:
pi.patches('xs','ys', source=geosource, fill_color=None,line_color="gray",line_width=0.5)
pf.patches('xs','ys', source=geosource, fill_color=None,line_color="gray",line_width=0.5)

GlyphRenderer(id='112651', ...)

In [183]:
pi.circle(x=instantei["x"],y=instantei["y"],size=instantei["radio"]/50,fill_color=instantei["color"],line_color="black",line_width=0.3)
pf.circle(x=instantef["x"],y=instantef["y"],size=instantef["radio"]/50,fill_color=instantef["color"],line_color="black",line_width=0.3)

GlyphRenderer(id='112663', ...)

In [184]:
pi.xaxis.axis_label = 'x'
pi.yaxis.axis_label = 'y'
pf.xaxis.axis_label = 'x'
pf.yaxis.axis_label = 'y'

In [185]:
fila = bly.row([pi,pf])

In [186]:
grid = bly.column([p0, fila])

In [187]:
bpl.show(grid)

In [188]:
export_png(grid, filename="../imagenes/s_" + sector + "_e_" + str(eta) +".png")

'd:\\proyectos\\rust\\dyn_cen_2\\imagenes\\s_cemp_mpio_e_1.png'